# Exercise: Multi-Modal Retrieval Augmented Generation from Scratch

## Objective

Implement a Retrieval Augmented Generation (RAG) system from scratch using the [PHI-3 vision](https://huggingface.co/microsoft/Phi-3-vision-128k-instruct) model, [Jina-CLIP-V1](https://huggingface.co/jinaai/jina-clip-v1) embeddings, and the Chroma vector database. Your task is to use this system to generate text based on the contents of the paper "Attention is All You Need" ([https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)).

## Allowed Libraries

You are only permitted to use the following libraries:
- `torch`
- `chromadb`
- `numpy`
- `io`
- `fitz`
- `requests`
- `PIL`
- `transformers`

## Components to Implement

1. **Document Ingestion**: Load the "Attention is All You Need" paper and prepare it for processing. This includes handling the PDF and extracting text and images as needed.

2. **Embeddings Generation**: Use Jina-CLIP-V1 to create embeddings for the text and images extracted from the document.

3. **Vector Database Management**: Utilize the Chroma vector database to store and manage the embeddings.

4. **Retrieval Mechanism**: Implement a mechanism to retrieve relevant document segments (text or images) based on a given query using the stored embeddings.

5. **Generation Model**: Integrate the PHI-3 vision model to generate new content based on the retrieved segments.

## Submission

Your submission should include:
- The complete code for the exercise.
- A README file explaining your implementation and how to run the code.
- Examples of generated outputs based on sample queries.

## Evaluation Criteria

- Correctness: The RAG system should accurately retrieve relevant document segments and generate coherent outputs.
- Efficiency: The implementation should be optimized for performance.
- Creativity: Innovative approaches to integrate and utilize the components are encouraged.
- Clarity: Code should be well-documented and easy to understand.

Good luck and enjoy the exercise!

Importing the required packages to load the file

In [1]:
!pip install transformers einops timm pillow
from transformers import AutoModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
model = AutoModel.from_pretrained('jinaai/jina-clip-v1', trust_remote_code=True)

Loading Jina-CLIP model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

configuration_clip.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- configuration_clip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

transform.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- transform.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


hf_model.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- hf_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- modeling_clip.py
- rope_embeddings.py
- eva_model.py
- transform.py
- hf_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

loading the file and extracting text and image from the document

In [3]:
 pip install PyMuPDF Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 90.3 MB/s eta 0:00:00


In [4]:
    import fitz
    from PIL import Image
    import io

In [17]:
import requests
import fitz
from PIL import Image
import io

pdf_url = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"
pdf_path = "attention_is_all_you_need.pdf" # Define a local path to save the PDF

# Download the PDF
response = requests.get(pdf_url)
with open(pdf_path, 'wb') as f:
    f.write(response.content)

# Open the downloaded PDF
pdf_document = fitz.open(pdf_path)

In [19]:
extracted_images = []

for page_number in range(pdf_document.page_count):
    page = pdf_document.load_page(page_number)
    image_list = page.get_images(full=True)

    for img_index, img_info in enumerate(image_list):
        xref = img_info[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        image = Image.open(io.BytesIO(image_bytes))
        extracted_images.append(image)

print(f"Extracted {len(extracted_images)} images.")

Extracted 3 images.


In [7]:
image_embeddings = model.encode_image(extracted_images)

preprocessor_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


processing_clip.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- processing_clip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


To generate multiple text embeddings, you can pass a list of text strings to the `model.encode_text()` function.

In [18]:
extracted_text = []

for page_number in range(pdf_document.page_count):
    page = pdf_document.load_page(page_number)
    extracted_text.append(page.get_text())

print(f"Extracted text from {len(extracted_text)} pages.")

Extracted text from 11 pages.


In [9]:
# Make sure to run the cell above to define the model
text_embeddings = model.encode_text(extracted_text)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

checking similarities between the embeddings

In [ ]:
print(text_embeddings[0] @ text_embeddings[1].T) # text embedding similarity
print(text_embeddings[0] @ image_embeddings[0].T) # text-image cross-modal similarity
print(text_embeddings[0] @ image_embeddings[1].T) # text-image cross-modal similarity
print(text_embeddings[1] @ image_embeddings[0].T) # text-image cross-modal similarity
print(text_embeddings[1] @ image_embeddings[1].T)# text-image cross-modal similarity

0.72897476
0.17103584
0.15112677
0.14756522
0.1411351


storing the embeddings in chromadb

In [11]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 20.4 MB/s

In [1]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs"
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [5]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "text_embeddings" # Renamed for clarity
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [6]:
# Use get_or_create_collection to handle cases where the collection might already exist
collection = client.get_or_create_collection(name=COLLECTION_NAME)
print(f"Accessed or created collection: {collection.name}")

NameError: name 'client' is not defined

In [24]:
# Make sure to run the cell above to define the model
if extracted_text:
    text_embeddings = model.encode_text(extracted_text)

    # Generate unique IDs for each text embedding
    text_embedding_ids = [f"text_embedding_{i}" for i in range(len(text_embeddings))]

    collection.add(
        embeddings=text_embeddings.tolist(),
        documents=extracted_text, # Include the original text content
        ids=text_embedding_ids
    )

    print(f"Successfully added {len(text_embeddings)} text embeddings to Chroma collection '{COLLECTION_NAME}'.")
else:
    print("No text extracted to create embeddings.")

Successfully added 11 text embeddings to Chroma collection 'text_embeddings'.


Now that the embeddings are stored in Chroma, we can implement the retrieval mechanism.

In [22]:
# Define a sample query
query_text = "What are the benefits of the Attention mechanism?"

# Generate embedding for the query using the Jina-CLIP model
query_embedding = model.encode_text(query_text)

# Query the Chroma collection
# We will query for the top 5 most relevant results (both text and images)
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['documents', 'distances', 'metadatas'] # Include relevant information in the results
)

# Display the results
print("Query Results:")
for i, (doc, dist, metadata) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
    print(f"Result {i+1} (Distance: {dist:.4f}):")
    if doc:
        print(f"  Document: {doc[:200]}...") # Print first 200 characters of text documents
    elif metadata and 'file_path' in metadata: # Assuming image metadata includes a file path or identifier
         print(f"  Image: {metadata['file_path']}") # Or some other image identifier
    print("-" * 20)

Query Results:
Result 1 (Distance: 1.0938):
--------------------
Result 2 (Distance: 1.1270):
--------------------
Result 3 (Distance: 1.1564):
--------------------
Result 4 (Distance: 1.1829):
--------------------
Result 5 (Distance: 1.2816):
--------------------


In [25]:
# Make sure to run the cell that loads the model first (cell 6Y5eyPDU4lIW)
if extracted_images:
    image_embeddings = model.encode_image(extracted_images)

    # Generate unique IDs for each image embedding
    image_embedding_ids = [f"image_embedding_{i}" for i in range(len(image_embeddings))]

    # Note: ChromaDB's current implementation doesn't directly support storing image objects in the 'documents' field.
    # We will store a placeholder or identifier for now. If image retrieval is crucial for generation,
    # you might need to store image paths or base64 encoded strings, or handle images separately.
    # For this example, we'll add a placeholder indicating an image is associated with the embedding.
    image_placeholders = [f"Image from page {extracted_images[i].info.get('page', 'unknown')}" for i in range(len(extracted_images))] # Assuming page info might be available


    collection.add(
        embeddings=image_embeddings.tolist(),
        documents=image_placeholders, # Add a placeholder for the image
        ids=image_embedding_ids
    )

    print(f"Successfully added {len(image_embeddings)} image embeddings to Chroma collection '{COLLECTION_NAME}'.")
else:
    print("No images extracted to create embeddings.")

Successfully added 3 image embeddings to Chroma collection 'text_embeddings'.


implementing the mechanism to generate document based on given query

In [ ]:
# Define a sample query
query_text = "What are the benefits of the Attention mechanism?"

# Generate embedding for the query using the Jina-CLIP model
query_embedding = model.encode_text(query_text)

# Query the Chroma collection
# We will query for the top 5 most relevant results (both text and images)
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['documents', 'distances', 'metadatas'] # Include relevant information in the results
)

# Display the results
print("Query Results:")
for i, (doc, dist, metadata) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
    print(f"Result {i+1} (Distance: {dist:.4f}):")
    if doc:
        print(f"  Document: {doc[:200]}...") # Print first 200 characters of text documents
    elif metadata and 'file_path' in metadata: # Assuming image metadata includes a file path or identifier
         print(f"  Image: {metadata['file_path']}") # Or some other image identifier
    print("-" * 20)

Query Results:
Result 1 (Distance: 1.7033):
--------------------
Result 2 (Distance: 1.7837):
--------------------
Result 3 (Distance: 1.8058):
--------------------


implement phi-3 model to generate new content based on generated output

In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.8/flash_attn-2.5.8+cu118torch2.3cxx11abiFALSE-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.1/122.1 MB 11.7 MB/s eta 0:00:00
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Attempting uninstall: flash-attn
    Found existing installation: flash_attn 2.8.3
    Uninstalling flash_attn-2.8.3:
      Successfully uninstalled flash_attn-2.8.3


In [7]:
from transformers import AutoModelForCausalLM
from transformers import  AutoProcessor
import torch

# Define the model ID for PHI-3 vision
model_id = "microsoft/Phi-3-vision-128k-instruct"

# Retrying to load the model with use_flash_attention_2=False
try:
    # Load the processor and the model, explicitly disabling FlashAttention2
    processor = AutoProcessor.from_pretrained(model_id, device_map="cuda", trust_remote_code=True)
    model_phi3 = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, trust_remote_code=True, use_flash_attention_2=False, attn_implementation="eager",device_map="auto", quantization_config=quantization_config)

    print(f"Loaded PHI-3 vision model: {model_id}")

except ImportError as e:
    print(f"Error loading model: {e}")
    print("It seems there is an issue with FlashAttention2. Please ensure you have the correct version installed and that it is compatible with your CUDA setup.")
    print("Refer to the transformers documentation on FlashAttention2 for more information.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

ImportError: /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [26]:
# Attempting to install flash_attn from source with specific CUDA flags
# This can sometimes resolve compatibility issues
import os

# You might need to set CUDA_HOME if it's not automatically detected
# os.environ['CUDA_HOME'] = '/usr/local/cuda' # Example path, verify in your environment

# Set the compute capability for your GPU
# You can find your GPU's compute capability by running !nvidia-smi and looking up your GPU model
# Replace '8.0' with your GPU's compute capability (e.g., '7.0' for V100, '7.5' for T4, '8.0' for A100, '8.6' for RTX 30 series, '8.9' for RTX 40 series)
# os.environ['TORCH_CUDA_ARCH_LIST'] = '8.0' # Example for A100

!pip install git+https://github.com/Dao-AILab/flash-attention.git --no-build-isolation

  Cloning https://github.com/Dao-AILab/flash-attention.git to /tmp/pip-req-build-ixtsid2u
  Running command git clone --filter=blob:none --quiet https://github.com/Dao-AILab/flash-attention.git /tmp/pip-req-build-ixtsid2u
  Resolved https://github.com/Dao-AILab/flash-attention.git to commit 91f14ca07b792645b72efbb05b233907a831c898
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp311-cp311-linux_x86_64.whl size=256022485 sha256=0abc62d04f28f140f4f76ab7cfd1d8ce24a69c6ab0cbace8d4ab99640b68dc0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n00zahiv/wheels/5d/d7/50/ea7b22955a7d70789a383640683f618c642c6c0cd343effe19
Successfully built flash_attn


In [ ]:
!pip install flash_attn==2.5.8 --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.1 MB/s eta 0:00:00


In [1]:
import torch

print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"PyTorch CUDA version: {torch.version.cuda}")
else:
    print("Please ensure you have a CUDA-enabled GPU and that CUDA is properly installed.")

CUDA is available: True
CUDA version: 12.4
PyTorch CUDA version: 12.4


In [2]:
# Reinstall flash_attn as a troubleshooting step for the ImportError
!pip install flash_attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.6 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp311-cp311-linux_x86_6

In [15]:
# Ensure the Jina-CLIP model is loaded
# Check if 'model' is defined, if not, load it
if 'model' not in globals():
    print("Loading Jina-CLIP model...")
    model = AutoModel.from_pretrained('jinaai/jina-clip-v1', trust_remote_code=True)
    print("Jina-CLIP model loaded.")
else:
    print("Jina-CLIP model is already loaded.")

Jina-CLIP model is already loaded.


In [23]:
# Define a sample query
query_text = "What are the benefits of the Attention mechanism?"

# Generate embedding for the query using the Jina-CLIP model
# Ensure the 'model' variable from earlier steps is available
query_embedding = model.encode_text(query_text)

# Query the Chroma collection
# Ensure 'collection' variable from earlier steps is available
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)

# Combine the retrieved text for the prompt
retrieved_text = "\n".join([doc for doc in results['documents'][0] if doc is not None])

# Check if any text was retrieved
if retrieved_text:
    # Create the prompt for the PHI-3 model
    messages = [
        {"role": "user", "content": f"Based on the following information:\n{retrieved_text}\n\nAnswer the question: {query_text}"}
    ]

    # Ensure 'processor' and 'model_phi3' variables from earlier steps are available
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(prompt, return_tensors="pt").to("cuda:0")

    generation_args = {
        "max_new_tokens": 500,
        "temperature": 0.0,
        "do_sample": False,
    }

    generate_ids = model_phi3.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

    # remove input tokens
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    print(response)
else:
    print("No relevant documents retrieved to generate a response.")

No relevant documents retrieved to generate a response.
